# Abstrakcija

Ko pišemo večje programe, je dobro, da jih razdelimo na manjše dele, ki jih lahko ločeno razumemo, razvijamo in preizkušamo. Pravimo, da programe pišemo _modularno_. V programskih jezikih modularnost dosežemo na več načinov. Eden, na katerega smo že navajeni, je razbitje kode na funkcije. Dostikrat pa želimo sorodne funkcije in podatke združiti v povezane enote.

V Pythonu to lahko storimo z razredi, ki združujejo določeno vrsto podatkov s funkcijami za delo na njih. Včasih je med seboj povezanih več razredov, ki jih združujemo v posamezne datoteke, ki jim pravimo tudi _[moduli](https://docs.python.org/3/tutorial/modules.html)_.

Tudi OCaml pozna module, ki imajo enako ime kot Pythonovi, a so precej naprednejši, saj omogočajo tudi skrivanje podrobnosti implementacije, čemur pravimo _abstrakcija_. Namen skrivanja podrobnosti seveda ni v zaščiti industrijskih skrivnosti, saj običajno delamo z lastnimi moduli, temveč v tem, da skrijemo podrobnosti in s tem poenostavimo razumevanje, preprečimo nepričakovano uporabo in olajšamo kasnejše spremembe implementacije.

## Moduli

OCamlovi moduli so zbirke definicij tipov, funkcij, vrednosti, (kasneje tudi drugih modulov), kot smo jih do sedaj pisali v datoteke ali v ukazno vrstico. V resnici vsaka `.ml` datoteka predstavlja modul, ki vsebuje vse definicije v njej. Do sedaj smo spoznali že nekaj modulov iz standardne knjižnice: `String` za delo z nizi, `List` za delo s seznami ali `Random` za delo z naključnimi vrednostmi.

Sestavimo svoj modul `Datum` za delo z datumi, v katerega za začetek naberimo funkcije in tipe, ki smo jih videli že prej. Module definiramo z ukazom `module`, vse definicije v modulu pa morajo biti znotraj bloka `struct ... end`. Glavni tip modula običajno poimenujemo `t`, da pišemo `Datum.t` namesto `Datum.datum`.

In [143]:
module Datum = struct
  type t = { dan : int; mesec : int; leto : int }

  let je_prestopno leto =
    (leto mod 4 = 0 && leto mod 100 <> 0) || leto mod 400 = 0
    
  let dolzina_meseca leto =
    function
    | 4 | 6 | 9 | 11 -> 30
    | 2 -> if je_prestopno leto then 29 else 28
    | _ -> 31

  let je_veljaven datum =
    let veljaven_dan = 1 <= datum.dan && datum.dan <= dolzina_meseca datum.leto datum.mesec
    and veljaven_mesec = 1 <= datum.mesec && datum.mesec <= 12
    in
    veljaven_dan && veljaven_mesec

  let naredi dan mesec leto =
    let datum = { dan; mesec; leto } in
    if je_veljaven datum then Some datum else None

  let to_string { dan; mesec; leto } =
    Format.sprintf "%04d-%02d-%02d" leto mesec dan
end

module Datum :
  sig
    type t = { dan : int; mesec : int; leto : int; }
    val je_prestopno : int -> bool
    val dolzina_meseca : int -> int -> int
    val je_veljaven : t -> bool
    val naredi : int -> int -> int -> t option
    val to_string : t -> string
  end


Do funkcij iz modula dostopamo prek `ImeModula.ime_funkcije`, tako kot smo do sedaj dostopali do funkcij iz modulov `List`, `String` in `Random`.

In [144]:
{ dan = 25; mesec = 6; leto = 1991} |> Datum.to_string

- : string = "1991-06-25"


In [145]:
Datum.dolzina_meseca 1991 6

- : int = 30


## Signature

Tako kot ima vsaka vrednost v OCamlu svoj tip, lahko zgoraj vidimo, da ga imajo tudi moduli. Tipom modulov pravimo _signature_. Signatura opisuje definirane tipe ter tipe definiranih vrednosti (ne pa njihovih implementacij). Signature pišemo podobno kot module, le da uporabimo blok `sig ... end`, tipe vrednosti pa podamo s ključno besedo `val`. Definicije tipov ostanejo enake.

### Definicije signatur

Signature definiramo podobno kot module, le da uporabimo ukaz `module type`.

In [146]:
module type DATUM =
  sig
    type t = { dan : int; mesec : int; leto : int; }
    val je_prestopno : int -> bool
    val dolzina_meseca : int -> int -> int
    val je_veljaven : t -> bool
    val naredi : int -> int -> int -> t option
    val to_string : t -> string
  end

module type DATUM =
  sig
    type t = { dan : int; mesec : int; leto : int; }
    val je_prestopno : int -> bool
    val dolzina_meseca : int -> int -> int
    val je_veljaven : t -> bool
    val naredi : int -> int -> int -> t option
    val to_string : t -> string
  end


Kot smo videli zgoraj, zna OCaml tako signaturo izračunati tudi sam. Toda tako kot smo morali prej nekaterim funkcijam z označbami sami vsiliti tip, bomo enako želeli s signaturami modulov. Razloga sta dva:
1. preverjanje skladnosti in
2. skrivanje implementacije.

### Preverjanje skladnosti implementacije

Prvi namen signatur je specifikacija vsebine modula. Običajno delo začnemo tako, da v signaturi opišemo, kaj bodo sestavni deli modula, nato pa začnemo pisati implementacijo, ki ji zadošča. Ko definiramo modul, lahko zraven z označbo podamo tudi njegovo signaturo:

In [147]:
module Datum : DATUM = struct
  type t = { dan : int; mesec : int; leto : int }

  let je_prestopno leto =
    (leto mod 4 = 0 && leto mod 100 <> 0) || leto mod 400 = 0
    
  let dolzina_meseca leto =
    function
    | 4 | 6 | 9 | 11 -> 30
    | 2 -> if je_prestopno leto then 29 else 28
    | _ -> 31

  let je_veljaven datum =
    let veljaven_dan = 1 <= datum.dan && datum.dan <= dolzina_meseca datum.leto datum.mesec
    and veljaven_mesec = 1 <= datum.mesec && datum.mesec <= 12
    in
    veljaven_dan && veljaven_mesec

  let naredi dan mesec leto =
    let datum = { dan; mesec; leto } in
    if je_veljaven datum then Some datum else None

  let to_string { dan; mesec; leto } =
    Format.sprintf "%04d-%02d-%02d" leto mesec dan
end

module Datum : DATUM


Isti signaturi lahko zadošča več modulov, ki se med seboj razlikujejo le v implementaciji. Na primer, tu je malo bolj ohlapna implementacija datumov. Seveda si take implementacije ne želimo, je pa morda dobro začetno izhodišče. Od vsega začetka razvoja pa bo OCaml preverjal, ali se implementacija ujema s signaturo.

In [148]:
module VednoVeljavenDatum : DATUM = struct
  type t = { dan : int; mesec : int; leto : int }

  let je_prestopno leto = false
    
  let dolzina_meseca _ _ = 31

  let je_veljaven _ = true

  let naredi dan mesec leto = Some { dan; mesec; leto }

  let to_string { dan; mesec; leto } =
    Format.sprintf "%04d-%02d-%02d" leto mesec dan
end

module VednoVeljavenDatum : DATUM


Če kakšna od naštetih funkcij manjka, bo OCaml to opazil in javil napako:

In [149]:
module Datum : DATUM = struct
  type t = { dan : int; mesec : int; leto : int }

  let je_prestopno leto =
    (leto mod 4 = 0 && leto mod 100 <> 0) || leto mod 400 = 0
    
  let dolzina_meseca leto =
    function
    | 4 | 6 | 9 | 11 -> 30
    | 2 -> if je_prestopno leto then 29 else 28
    | _ -> 31

  let je_veljaven datum =
    let veljaven_dan = 1 <= datum.dan && datum.dan <= dolzina_meseca datum.leto datum.mesec
    and veljaven_mesec = 1 <= datum.mesec && datum.mesec <= 12
    in
    veljaven_dan && veljaven_mesec

  let naredi dan mesec leto =
    let datum = { dan; mesec; leto } in
    if je_veljaven datum then Some datum else None
end

error: compile_error

Podobno bo preveril, ali se pri vseh definicijah ujemajo tipi.

In [150]:
module Datum : DATUM = struct
  type t = { dan : int; mesec : int; leto : int }

  let je_prestopno leto =
    (leto mod 4 = 0 && leto mod 100 <> 0) || leto mod 400 = 0
    
  let dolzina_meseca leto =
    function
    | 4 | 6 | 9 | 11 -> 30
    | 2 -> if je_prestopno leto then 29 else 28
    | _ -> 31

  let je_veljaven datum =
    let veljaven_dan = 1 <= datum.dan && datum.dan <= dolzina_meseca datum.leto datum.mesec
    and veljaven_mesec = 1 <= datum.mesec && datum.mesec <= 12
    in
    veljaven_dan && veljaven_mesec

  let naredi (dan, mesec, leto) =
    let datum = { dan; mesec; leto } in
    if je_veljaven datum then Some datum else None

  let to_string { dan; mesec; leto } =
    Format.sprintf "%04d-%02d-%02d" leto mesec dan
end

error: compile_error

### Skrivanje implementacije

Glavna prednost uporabe signatur pa je v tem, da lahko z njimi implementacij ne le preverjamo, temveč tudi deloma skrivamo. Če uporabljamo pomožno funkcijo, ki ni del signature, navzven ne bo vidna. Na primer, funkcije za izračun veljavnosti datuma lahko skrijemo.

In [151]:
module type DATUM =
  sig
    type t = { dan : int; mesec : int; leto : int; }
    val naredi : int -> int -> int -> t option
    val to_string : t -> string
  end

module type DATUM =
  sig
    type t = { dan : int; mesec : int; leto : int; }
    val naredi : int -> int -> int -> t option
    val to_string : t -> string
  end


In [152]:
module Datum : DATUM = struct
  type t = { dan : int; mesec : int; leto : int }

  let je_prestopno leto =
    (leto mod 4 = 0 && leto mod 100 <> 0) || leto mod 400 = 0
    
  let dolzina_meseca leto =
    function
    | 4 | 6 | 9 | 11 -> 30
    | 2 -> if je_prestopno leto then 29 else 28
    | _ -> 31

  let je_veljaven datum =
    let veljaven_dan = 1 <= datum.dan && datum.dan <= dolzina_meseca datum.leto datum.mesec
    and veljaven_mesec = 1 <= datum.mesec && datum.mesec <= 12
    in
    veljaven_dan && veljaven_mesec

  let naredi dan mesec leto =
    let datum = { dan; mesec; leto } in
    if je_veljaven datum then Some datum else None

  let to_string { dan; mesec; leto } =
    Format.sprintf "%04d-%02d-%02d" leto mesec dan
end

module Datum : DATUM


Modul še vedno vsebuje vse funkcije iz signature, zato se OCaml ne pritoži. A če poskusimo dostopati do dodatnih funkcij, bomo dobili napako:

In [153]:
Datum.naredi 25 6 1991

- : Datum.t option = Some {Datum.dan = 25; mesec = 6; leto = 1991}


In [154]:
Datum.dolzina_meseca 1991 6

error: compile_error

Skrivanje implementacij uporabnikom poenostavi uporabo, saj izpostavi le ključne funkcije. Hkrati pa razvijalcem olajša kasnejše spremembe implementacije, če na primer najdejo boljši algoritem. Če pomožne funkcije ne bi bile skrite, bi se lahko nanje kdo zanašal, kar bi otežilo kasnejše spremembe.

### Abstraktni tipi

Poleg pomožnih funkcij lahko skrivamo tudi definicije tipov. To ne poenostavlja samo uporabe in kasnejših razširitev, temveč tudi zagotavlja pravilnost podatkov. Recimo, kljub temu, da smo pripravili funkcijo `naredi`, ki bo vedno ustvarila veljaven datum, lahko uporabnik še vedno ustvari neveljaven datum.

In [155]:
{ dan = 32; mesec = 13; leto = 2024 } |> Datum.to_string

- : string = "2024-13-32"


Temu se lahko izgonemo tako, da skrijemo definicijo tipa, samo povemo, da obstaja.

In [156]:
module type DATUM =
  sig
    type t
    val naredi : int -> int -> int -> t option
    val to_string : t -> string
  end

module type DATUM =
  sig
    type t
    val naredi : int -> int -> int -> t option
    val to_string : t -> string
  end


In [157]:
module Datum : DATUM = struct
  type t = { dan : int; mesec : int; leto : int }

  let je_prestopno leto =
    (leto mod 4 = 0 && leto mod 100 <> 0) || leto mod 400 = 0
    
  let dolzina_meseca leto =
    function
    | 4 | 6 | 9 | 11 -> 30
    | 2 -> if je_prestopno leto then 29 else 28
    | _ -> 31

  let je_veljaven datum =
    let veljaven_dan = 1 <= datum.dan && datum.dan <= dolzina_meseca datum.leto datum.mesec
    and veljaven_mesec = 1 <= datum.mesec && datum.mesec <= 12
    in
    veljaven_dan && veljaven_mesec

  let naredi dan mesec leto =
    let datum = { dan; mesec; leto } in
    if je_veljaven datum then Some datum else None

  let to_string { dan; mesec; leto } =
    Format.sprintf "%04d-%02d-%02d" leto mesec dan
end

module Datum : DATUM


Sedaj je edini način, da ustvarimo vrednosti tipa `Datum.t` ta, da pokličemo funkcijo naredi, ki preveri veljavnost. Tudi če uporabnik uporabi identični tip, kot je v implementaciji, bo OCaml preprečil neposredno manipulacijo z njim.

In [158]:
{ dan = 32; mesec = 13; leto = 2024 } |> Datum.to_string

error: compile_error

Poleg tega lahko uporabimo tudi drugačno implementacijo datumov, recimo da mesece predstavimo z naštevnim tipom.

## Primer: štetje različnih elementov

Za primer izračunajmo, koliko različnih elementov vsebuje dani seznam. Ena izmed možnosti je, da se sprehajamo čez seznam ter beležimo seznam elementov, ki smo jih že videli, začenši s praznim seznamom. Vsak element primerjamo z že videnimi in če ga še nismo videli, ga dodamo v seznam. Seveda bo ta primer majhen in ga ne bi bilo treba razčlenjevati, a bomo to vseeno storili, da spoznamo pristop.

In [159]:
let stevilo_razlicnih xs =
  let rec aux ze_videni = function
    | [] -> List.length ze_videni
    | x :: xs ->
        if List.mem x ze_videni
        then aux ze_videni xs
        else aux (x :: ze_videni) xs
  in
  aux [] xs

val stevilo_razlicnih : 'a list -> int = <fun>


In [160]:
stevilo_razlicnih [1; 2; 1; 2; 1; 2; 3]

- : int = 3


Napišimo še nekaj funkcij, s katerimi bomo izmerili (ne)učinkovitost take implementacije.

In [161]:
let nakljucni_seznam m n = List.init n (fun _ -> Random.int m)

val nakljucni_seznam : int -> int -> int list = <fun>


In [162]:
nakljucni_seznam 5 20

- : int list = [0; 3; 3; 1; 1; 3; 4; 3; 0; 2; 1; 1; 4; 3; 1; 1; 1; 2; 0; 4]


In [163]:
stevilo_razlicnih @@ nakljucni_seznam 5 20

- : int = 5


In [164]:
let seznam_zaporednih n = List.init n (fun i -> i)

val seznam_zaporednih : int -> int list = <fun>


In [165]:
seznam_zaporednih 10

- : int list = [0; 1; 2; 3; 4; 5; 6; 7; 8; 9]


In [166]:
stevilo_razlicnih @@ seznam_zaporednih 10

- : int = 10


In [167]:
let stopaj f x =
  let zacetek = Sys.time () in
  let y = f x in
  let konec = Sys.time () in
  let izpis = 
    Printf.sprintf "Porabljen čas: %f ms\n" (1000. *. (konec -. zacetek))
  in
  print_endline izpis;
  y

val stopaj : ('a -> 'b) -> 'a -> 'b = <fun>


In [168]:
stopaj stevilo_razlicnih (seznam_zaporednih 1000)

Porabljen čas: 9.122000 ms



- : int = 1000


In [169]:
stopaj stevilo_razlicnih (seznam_zaporednih 2000)

Porabljen čas: 36.560000 ms



- : int = 2000


Za dvakrat daljši seznam smo potrebovali okoli štirikrat več časa, saj se mora funkcija `List.mem` sprehajati po vedno daljšem seznamu, da ugotovi, da elementa ni v njem. Razlog za neučinkovitost je v tem, da za beleženje videnih elemente uporabljamo sezname, čeprav potrebujemo samo množice, ki se ne ozirajo na vrstni red in število ponovitev. V kratkem bomo spoznali učinkovite podatkovne strukture za predstavitev množic, zaenkrat pa si pripravimo teren za spremembe implementacij.

In [170]:
module type MNOZICA = sig
  type 'a t
  val prazna : 'a t
  val dodaj : 'a -> 'a t -> 'a t
  val velikost : 'a t -> int
  val vsebuje : 'a -> 'a t -> bool
end

module type MNOZICA =
  sig
    type 'a t
    val prazna : 'a t
    val dodaj : 'a -> 'a t -> 'a t
    val velikost : 'a t -> int
    val vsebuje : 'a -> 'a t -> bool
  end


In [171]:
module Mnozica : MNOZICA = struct
  type 'a t = 'a list
  let prazna = []
  let velikost m = List.length m
  let vsebuje x m = List.mem x m
  let dodaj x m = if vsebuje x m then m else x :: m
end

module Mnozica : MNOZICA


Na ta način naš algoritem namesto kot

In [172]:
let stevilo_razlicnih xs =
  let rec aux ze_videni = function
    | [] -> List.length ze_videni
    | x :: xs ->
        if List.mem x ze_videni
        then aux ze_videni xs
        else aux (x :: ze_videni) xs
  in
  aux [] xs

val stevilo_razlicnih : 'a list -> int = <fun>


napišemo kot:

In [173]:

let stevilo_razlicnih xs =
  let rec aux ze_videni = function
    | [] -> Mnozica.velikost ze_videni
    | x :: xs -> aux (Mnozica.dodaj x ze_videni) xs
  in
  aux Mnozica.prazna xs

val stevilo_razlicnih : 'a list -> int = <fun>


Vidimo, da je definicija precej bolj pregledna, saj smo implementacijo in uporabo množic razdelili na dva dela.